This file is taken from Peter Yoachim github at https://github.com/yoachim/20_Scratch/blob/master/maf_examples/read_example.ipynb

In [1]:
%matplotlib inline
import numpy as np
import matplotlib.pyplot as plt
import healpy as hp

import lsst.sims.maf.db as db
import lsst.sims.maf.utils as utils
import lsst.sims.maf.metrics as metrics
import lsst.sims.maf.slicers as slicers
import lsst.sims.maf.stackers as stackers
import lsst.sims.maf.plots as plots
import lsst.sims.maf.metricBundles as metricBundles

In [2]:
# Connect to any old simulation



In [3]:
import glob 
dbDir = '/home/idies/workspace/lsst_cadence/FBS_1.7/'
DBPATH_V1_5 = dbDir
COLS = 'night, observationStartMJD, fieldRA, fieldDec, filter, proposalId, note'
NSIDE = 64

runs = glob.glob(dbDir+"baseline*")#+"footprint*")
runs.sort()

In [4]:
runs = [i.split('/')[-1].split('.d')[0] for i in runs]
# runs = [i.split('/')[-1].split('.d')[0] for i in runs if i.split('/')[-1].split('.d')[0].split('_')[3]=='noddf']


# runs = [i.split('/')[-1].split('.d')[0] for i in runs if not i.split('/')[-1].split('.d')[0].startswith('footprint_gp')]

In [5]:
runs

['baseline_nexp1_v1.7_10yrs', 'baseline_nexp2_v1.7_10yrs']

In [9]:
import sys
import os
sys.path.insert(0, '../sims_maf_contrib/')
from mafContrib import NumObsMetric

for run in runs:
    
    run_name = run #'bulges_i_heavy_v1.5_10yrs'
    dbFile = dbDir+ run_name+ '.db'
    conn = db.OpsimDatabase(dbFile)
    your_username = 'somayeh91'
    outDir = '/home/idies/workspace/Storage/{}/persistent/MAFOutput/Results_single_opsim_on_GP/'.format(your_username)
    resultsDb = db.ResultsDb(outDir=outDir)

    # Let's compute the coadded depth on the whole sky, and then the median
    bundleList = []
    # sql = 'filter="r"'
    metric = NumObsMetric()
    # slicer = slicers.HealpixSlicer()
    slicer = GP_slicer_selector(run_name+'.db')
    summaryStats = [metrics.MedianMetric(), metrics.SumMetric(), metrics.MinMetric()]
    # plotDict = {'colorMax': 27.4}
    bundleList.append(metricBundles.MetricBundle(metric,slicer, summaryMetrics=summaryStats))#, plotDict=plotDict

    bd = metricBundles.makeBundlesDictFromList(bundleList)
    # bd[run_name+'_' +('opsim_NumObsMetric_USER').split('m_')[1]] = bd.pop('opsim_NumObsMetric_USER')
    bg = metricBundles.MetricBundleGroup(bd, conn, outDir=outDir, resultsDb=resultsDb)
    bg.runAll()

    os.rename(r'./../MAFOutput/Results_single_opsim_on_GP/opsim_NumObsMetric_USER.npz','./../MAFOutput/Results_single_opsim_on_GP/'+str(run_name)+r'_NumObsMetric_USER.npz')


connect to  /home/idies/workspace/lsst_cadence/FBS_1.7/baseline_nexp1_v1.7_10yrs.db 
 SELECT night, observationStartMJD, fieldRA, fieldDec, filter, proposalId, note FROM SummaryAllProps
Querying database SummaryAllProps with no constraint for columns ['fieldRA', 'night', 'fieldDec'].
Found 2208619 visits
Running:  ['opsim_NumObsMetric_USER']
Completed metric generation.
Running reduce methods.
Running summary statistics.
Completed.
connect to  /home/idies/workspace/lsst_cadence/FBS_1.7/baseline_nexp2_v1.7_10yrs.db 
 SELECT night, observationStartMJD, fieldRA, fieldDec, filter, proposalId, note FROM SummaryAllProps
Querying database SummaryAllProps with no constraint for columns ['fieldRA', 'night', 'fieldDec'].
Found 2045493 visits
Running:  ['opsim_NumObsMetric_USER']
Completed metric generation.
Running reduce methods.
Running summary statistics.
Completed.


In [18]:
bd

{'bulges_bs_v1.5_10yrsNumObsMetric_USER': <lsst.sims.maf.metricBundles.metricBundle.MetricBundle at 0x7f21f4e392b0>}

In [9]:
bg.plotAll(closefigs=False)

Plotting figures with "" constraint now.
Plotting complete.


In [10]:
# Plot both -- we'll start using the PlotHandler because it's a bit easier to update plot parameters
ph = plots.PlotHandler(outDir='test', figformat='png', thumbnail=False, savefig=False)

# Just plot the sky map
plotFunc = plots.HealpixSkyMap()
plotDict = {'colorMin': 1, 'colorMax':1050, 'scale': 'log'}

# for r in runs:
ph.setMetricBundles(bd)
ph.plot(plotFunc=plotFunc, plotDicts=plotDict)

3

In [6]:
def GP_slicer_selector (dbname, COLS = 'night, observationStartMJD, fieldRA, fieldDec, filter, proposalId, note',\
                       NSIDE = 64, DBPATH_V1_5 = dbDir):
    #dbname =  'footprint_add_mag_cloudsv1.5_10yrs.db'

    # get some data from opsim database
    df_opsim = get_cols(dbfile=DBPATH_V1_5 + dbname, cols=COLS)

    # get healpix Ids
    df_opsim = get_pixId(df_opsim, nside=NSIDE)

    # in galactic coordinates
    df_opsim['fieldGl'], df_opsim['fieldGb'] = radec2gal(ra=df_opsim['fieldRA'].values, dec=df_opsim['fieldDec'].values)

    # define the shape of galactic plane
    peakDeg=15.
    taperDeg=90.
    constWidth=.5

    # get the latitude of galaxy region
    df_opsim['galaxyb'] = getAvoidanceLatitudes(df_opsim['fieldGl'].values, peakDeg, taperDeg, constWidth)

    # check if a field is ingalaxy
    df_opsim['ingalaxy'] = ( df_opsim['fieldGb'] > -df_opsim['galaxyb'] ) & ( df_opsim['fieldGb'] < df_opsim['galaxyb'] ) 

    # get pixId in galactic plane
    ipix_GP = df_opsim.pixId[df_opsim.ingalaxy].unique()
    
    ra, dec = pix2radec(nside=64, ipix=ipix_GP)
    slicer = slicers.UserPointsSlicer(ra, dec, latLonDeg=True, badval=0)


    
    return slicer





In [7]:
def getAvoidanceLatitudes(galL, peakDeg=10., taperDeg=80., constWidth=5.):
    
    """Returns the (positive) GP avoidance region for input galactic longitude
    Input:
        galL: in degree
        peakDeg: galaxy peak size
        taperDeg: half width, in degree
        constWidth: in degree
    return 
        val: the positive galaxy b, in degree
    """  
    # The following is adapted from spatialPlotters.py in sims_maf, method _plot_mwZone
    
    # astropy uses 0 <= glon <= 2pi, so we shift the input values accordingly.
    galL = np.radians(galL)
    galL_use = np.copy(galL)
    gSec = np.where(galL_use > np.pi)
    galL_use[gSec] -= 2.0 * np.pi
    
    peakWidth=np.radians(peakDeg)
    taperLength=np.radians(taperDeg)
    val = peakWidth * np.cos(galL_use / taperLength * np.pi / 2.)
    
    # Remove the cosine peak at anticenter
    gFar = np.where(np.abs(galL_use) > taperLength)[0]
    val[gFar] = 0.

    val += np.radians(constWidth)
    
    return np.rad2deg(val)

In [8]:
#  get columns from databases
def get_cols(dbfile, cols='night', lim=None):
    """
    cols = 'night, observationStartMJD, fieldRA, fieldDec, filter, proposalId, note'
    connect to database
    Returns: a pandas DataFrame
    
    Examples:
    
    cols = 'night, observationStartMJD, fieldRA, filter, fieldDec, proposalId, note, '

    df = get_cols(dbfile=dbpath + db, cols=cols, lim='night<300')
   
    """
    import sqlite3
    import pandas as pd
    connection = sqlite3.connect(dbfile)
    cursor = connection.cursor()
    
    if lim!=None:
        sqlstr = "SELECT {} FROM SummaryAllProps where {}".format(cols, lim)
    else:
        sqlstr = "SELECT {} FROM SummaryAllProps".format(cols)

    print('connect to ', dbfile, '\n', sqlstr)
    cursor.execute(sqlstr)
    data = cursor.fetchall()
    
    # convert data to dataframe
    data_df = pd.DataFrame(data, columns=cols.split(', '))
    
    # close connection
    connection.close()
    
    return data_df

def get_pixId(df, nside=16):
    '''get pixId for each field'''
    pixId = RADec2pix(nside, ra=df['fieldRA'].values, dec=df['fieldDec'].values)
    df['pixId'] = pixId
    return df

def pix2radec(nside, ipix, degree=True):
    """from ipix to ra/dec"""
    
    theta, phi = hp.pix2ang(nside, ipix)
    ra = phi
    dec = np.pi/2. - theta
    if degree:
        ra = np.rad2deg(ra)
        dec = np.rad2deg(dec)
    return ra, dec

def RADec2pix(nside, ra, dec, degree=True):
    """
    Calculate the nearest healpixel ID of an RA/Dec array, assuming nside.

    Parameters
    ----------
    nside : int
        The nside value of the healpix grid.
    ra : numpy.ndarray
        The RA values to be converted to healpix ids, in degree by default.
    dec : numpy.ndarray
        The Dec values to be converted to healpix ids, in degree by default.

    Returns
    -------
    numpy.ndarray
        The healpix ids.
    """
    if degree:
        ra = np.radians(ra) # change to radians
        dec = np.radians(dec)
    
    lat = np.pi/2. - dec
    hpid = hp.ang2pix(nside, lat, ra )
    return hpid

def radec2xyz(ra, dec):
    '''convert ra/dec to xyz'''
    theta = 0.5 * np.pi - np.deg2rad(dec)
    phi = np.deg2rad(ra)
    xyz = hp.ang2vec(theta, phi)
    
    return xyz

def radec2gal(ra, dec):
    '''convert from ra/dec to galactic l/b'''
    from astropy.coordinates import SkyCoord
    from astropy import units as u
    c = SkyCoord(ra=ra, dec=dec, 
                 unit=(u.degree, u.degree))
    
    l = c.galactic.l.degree
    b = c.galactic.b.degree
    return l, b

def gal2radec(gl, gb):
    '''convert gl/gb to ra/dec'''
    from astropy.coordinates import SkyCoord
    from astropy import units as u
    c = SkyCoord(l=gl, b=gb, frame="galactic",
                unit=(u.degree, u.degree))
    ra = c.icrs.ra.degree
    dec = c.icrs.dec.degree
    return ra, dec


# plot function
# plot skymap

def plot_mwd(ax, RA, Dec, c, org=0, title='Mollweide projection', projection='mollweide', vmax=None, colorbar=True, **kwargs):
    ''' RA, Dec are arrays of the same length.
    RA takes values in [0,360), Dec in [-90,90],
    which represent angles in degrees.
    org is the origin of the plot, 0 or a multiple of 30 degrees in [0,360).
    title is the title of the figure.
    projection is the kind of projection: 'mollweide', 'aitoff', 'hammer', 'lambert'
    
    Examples:
    >>> fig, ax = plt.subplots(1, 1, figsize=(8, 8), 
                            subplot_kw={'projection': 'mollweide'}); 
        plot_mwd(ax, x, y, c, title='') 
        
    '''
    import numpy as np
    import matplotlib.pyplot as plt

    if vmax is None:
        vmax = c.max()
    
    x = np.remainder(RA+360-org,360) # shift RA values
    ind = x>180
    x[ind] -=360    # scale conversion to [-180, 180]
    x=-x    # reverse the scale: East to the left
    tick_labels = np.array([150, 120, 90, 60, 30, 0, 330, 300, 270, 240, 210])
    tick_labels = np.remainder(tick_labels+360+org,360)
    # fig = plt.figure(figsize=(10, 5))
    # ax = fig.add_subplot(111, projection=projection, facecolor ='LightCyan')
    # ax = fig.add_subplot(111, projection=projection, facecolor ='honeydew')
    sc = ax.scatter(np.radians(x), np.radians(Dec), c=c, marker='o', vmin=0, vmax=vmax, **kwargs)  # convert degrees to radians
    # ax.set_xticklabels(tick_labels)     # we add the scale on the x axis
    ax.set_title(title)
    ax.title.set_fontsize(15)
    ax.set_xlabel("RA")
    ax.xaxis.label.set_fontsize(12)
    ax.set_ylabel("Dec")
    ax.yaxis.label.set_fontsize(12)
    ax.grid(True)
    if colorbar:
        cbar = plt.colorbar(sc, ax=ax, orientation="horizontal")
    #cbar.set_clim(-2.0, 2.0)

def plot_filter_Nv(df, coord='C', figsize= (12, 6), colorbar=True, vmax=None, **kwargs):
    """plot number of visits in 6 filters"""
    
    fig, axs = plt.subplots(2, 3, figsize=figsize,
                            subplot_kw={'projection': 'mollweide'}); 
    axs = axs.ravel()
    FILTERS = ['u', 'g', 'r', 'i', 'z', 'y']
    for i, f in enumerate(FILTERS):
        
        df_f = df[ df['filter']==f ]
        if coord=='G':
            x = df_f['fieldGl'].values
            y = df_f['fieldGb'].values       
        else:
            x = df_f['fieldRA'].values
            y = df_f['fieldDec'].values
        c = df_f['Nv'].values
        if len(c)!=0:
            plot_mwd(axs[i], x, y, c, title=f, colorbar=colorbar, vmax=vmax, **kwargs)
        axs[i].set_xticklabels([])
        axs[i].set_yticklabels([])
        axs[i].set_title(f)


